In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class MetaTradeAnalyzer:
    def __init__(self, years=5):
        self.years = years
        self.end_date = datetime.now()
        self.start_date = self.end_date - timedelta(days=years*365)
        self.data = None
        self.signals = None
        self.current_position = None

    def fetch_data(self):
        """Obtiene datos de META y QQQ"""
        try:
            meta = yf.download('META', start=self.start_date, end=self.end_date)
            qqq = yf.download('QQQ', start=self.start_date, end=self.end_date)

            if len(meta) == 0 or len(qqq) == 0:
                raise ValueError("No se pudieron obtener datos para META o QQQ")

            common_idx = meta.index.intersection(qqq.index)
            meta = meta.loc[common_idx]
            qqq = qqq.loc[common_idx]

            self.data = pd.DataFrame(index=common_idx)
            self.data['META_Close'] = meta['Adj Close']
            self.data['QQQ_Close'] = qqq['Adj Close']
            self.data['META_Volume'] = meta['Volume']
            self.data['META_High'] = meta['High']
            self.data['META_Low'] = meta['Low']

            self.data['META_Returns'] = self.data['META_Close'].pct_change()
            self.data['QQQ_Returns'] = self.data['QQQ_Close'].pct_change()

            print(f"Datos cargados exitosamente. Rango de fechas: {self.data.index[0]} a {self.data.index[-1]}")

        except Exception as e:
            print(f"Error al obtener datos: {str(e)}")
            raise

    def calculate_stop_loss(self, window=20, confidence_level=0.95):
        """Calcula stop loss dinámico y señales"""
        if self.data is None:
            raise ValueError("Debe ejecutar fetch_data() primero")

        try:
            meta_returns = self.data['META_Returns'].dropna()
            qqq_returns = self.data['QQQ_Returns'].dropna()
            beta = np.cov(meta_returns, qqq_returns)[0,1] / np.var(qqq_returns)

            stops, distances, signals = [], [], []
            position = 1  # 1: Long, 0: Fuera del mercado

            for i in range(window, len(self.data)):
                window_prices = self.data['META_Close'].iloc[i-window:i]
                returns = np.log(window_prices[1:] / window_prices[:-1])
                sigma = np.std(returns)
                var = stats.norm.ppf(1 - confidence_level, loc=np.mean(returns), scale=sigma)

                current_price = self.data['META_Close'].iloc[i]
                stop_loss = current_price - (beta * (1/sigma) * (1/abs(var))) if sigma > 0 and var != 0 else current_price * 0.95

                if position == 1 and current_price <= stop_loss:
                    position = 0
                    signal = -1
                elif position == 0 and current_price > stop_loss * 1.02 and self.data['META_Returns'].iloc[i-5:i].mean() > 0:
                    position = 1
                    signal = 1
                else:
                    signal = 0

                stops.append(stop_loss)
                distances.append((current_price - stop_loss) / current_price * 100)
                signals.append(signal)

            self.data = self.data.iloc[window:].copy()
            self.data['Stop_Loss'] = stops
            self.data['Distance_Percent'] = distances
            self.data['Signal'] = signals
            self.current_position = position

            print("Stop loss y señales calculados exitosamente")

        except Exception as e:
            print(f"Error al calcular stop loss: {str(e)}")
            raise

    def plot_analysis(self):
        """Crea gráficos interactivos con Plotly"""
        if self.data is None:
            raise ValueError("No hay datos para graficar")

        try:
            fig = make_subplots(
                rows=3, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.05,
                subplot_titles=('Precio y Stop Loss', 'Distancia al Stop Loss (%)', 'Volumen'),
                row_heights=[0.5, 0.25, 0.25]
            )

            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['META_Close'], name='META Price', line=dict(color='blue')), row=1, col=1)
            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Stop_Loss'], name='Stop Loss', line=dict(color='red', dash='dash')), row=1, col=1)

            buy_signals = self.data[self.data['Signal'] == 1]
            sell_signals = self.data[self.data['Signal'] == -1]

            if not buy_signals.empty:
                fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['META_Close'], name='Buy Signal', mode='markers', marker=dict(color='green', size=10, symbol='triangle-up')), row=1, col=1)

            if not sell_signals.empty:
                fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['META_Close'], name='Sell Signal', mode='markers', marker=dict(color='red', size=10, symbol='triangle-down')), row=1, col=1)

            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Distance_Percent'], name='Distance to Stop Loss', line=dict(color='orange')), row=2, col=1)
            fig.add_trace(go.Bar(x=self.data.index, y=self.data['META_Volume'], name='Volume', marker_color='lightblue'), row=3, col=1)

            fig.update_layout(height=1000, title_text="META Analysis Dashboard", showlegend=True)
            return fig

        except Exception as e:
            print(f"Error al crear gráficos: {str(e)}")
            raise

    def get_current_signals(self):
        """Obtiene señales actuales y métricas relevantes"""
        if self.data is None:
            raise ValueError("No hay datos para analizar")

        current_price = self.data['META_Close'].iloc[-1]
        current_stop = self.data['Stop_Loss'].iloc[-1]
        current_distance = self.data['Distance_Percent'].iloc[-1]

        metrics = {
            'Precio Actual': f"${current_price:.2f}",
            'Stop Loss': f"${current_stop:.2f}",
            'Distancia al Stop (%)': f"{current_distance:.2f}%",
            'Posición Actual': 'LONG' if self.current_position == 1 else 'FUERA',
            'Señal': self.get_signal_description(),
            'Volatilidad 20d': f"{self.data['META_Returns'].tail(20).std() * np.sqrt(252) * 100:.2f}%",
            'Tendencia 5d': 'ALCISTA' if self.data['META_Returns'].tail(5).mean() > 0 else 'BAJISTA'
        }

        return metrics

    def get_signal_description(self):
        """Genera descripción detallada de la señal actual"""
        if self.data is None:
            raise ValueError("No hay datos para analizar")

        last_signal = self.data['Signal'].iloc[-1]
        if last_signal == 1:
            return "COMPRAR: Precio por encima del stop loss con tendencia positiva"
        elif last_signal == -1:
            return "VENDER: Precio ha tocado el stop loss"
        else:
            return "MANTENER: Stop loss no activado" if self.current_position == 1 else "ESPERAR: Condiciones de entrada no cumplidas"

# Ejemplo de uso
def main():
    try:
        print("Iniciando análisis de META...")
        analyzer = MetaTradeAnalyzer(years=5)

        print("Descargando datos...")
        analyzer.fetch_data()

        print("Calculando señales...")
        analyzer.calculate_stop_loss()

        print("\n=== Señales Actuales ===")
        current_signals = analyzer.get_current_signals()
        for metric, value in current_signals.items():
            print(f"{metric}: {value}")

        print("\nGenerando gráficos...")
        fig = analyzer.plot_analysis()
        fig.show()

    except Exception as e:
        print(f"Error en la ejecución principal: {str(e)}")

if __name__ == "__main__":
    main()

Iniciando análisis de META...
Descargando datos...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Datos cargados exitosamente. Rango de fechas: 2019-11-05 00:00:00+00:00 a 2024-11-01 00:00:00+00:00
Calculando señales...
Stop loss y señales calculados exitosamente

=== Señales Actuales ===
Precio Actual: $567.16
Stop Loss: $538.80
Distancia al Stop (%): 5.00%
Posición Actual: LONG
Señal: MANTENER: Stop loss no activado
Volatilidad 20d: 25.02%
Tendencia 5d: BAJISTA

Generando gráficos...


In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class MetaTradeAnalyzer:
    def __init__(self, years=5):
        self.years = years
        self.end_date = datetime.now()
        self.start_date = self.end_date - timedelta(days=years*365)
        self.data = None
        self.signals = None
        self.current_position = None

    def fetch_data(self):
        """Obtiene datos de META y QQQ"""
        try:
            meta = yf.download('META', start=self.start_date, end=self.end_date)
            qqq = yf.download('QQQ', start=self.start_date, end=self.end_date)

            if len(meta) == 0 or len(qqq) == 0:
                raise ValueError("No se pudieron obtener datos para META o QQQ")

            common_idx = meta.index.intersection(qqq.index)
            meta = meta.loc[common_idx]
            qqq = qqq.loc[common_idx]

            self.data = pd.DataFrame(index=common_idx)
            self.data['META_Close'] = meta['Adj Close']
            self.data['QQQ_Close'] = qqq['Adj Close']
            self.data['META_Volume'] = meta['Volume']
            self.data['META_High'] = meta['High']
            self.data['META_Low'] = meta['Low']

            self.data['META_Returns'] = self.data['META_Close'].pct_change()
            self.data['QQQ_Returns'] = self.data['QQQ_Close'].pct_change()

            print(f"Datos cargados exitosamente. Rango de fechas: {self.data.index[0]} a {self.data.index[-1]}")

        except Exception as e:
            print(f"Error al obtener datos: {str(e)}")
            raise

    def calculate_stop_loss_and_take_profit(self, window=20, confidence_level=0.95):
        """Calcula stop loss y take profit dinámicos y señales"""
        if self.data is None:
            raise ValueError("Debe ejecutar fetch_data() primero")

        try:
            meta_returns = self.data['META_Returns'].dropna()
            qqq_returns = self.data['QQQ_Returns'].dropna()
            beta = np.cov(meta_returns, qqq_returns)[0,1] / np.var(qqq_returns)

            stops, take_profits, distances, signals = [], [], [], []
            position = 1  # 1: Long, 0: Fuera del mercado

            for i in range(window, len(self.data)):
                window_prices = self.data['META_Close'].iloc[i-window:i]
                returns = np.log(window_prices[1:] / window_prices[:-1])
                sigma = np.std(returns)

                # Cálculo del VaR al 99%
                var_99 = stats.norm.ppf(0.99, loc=np.mean(returns), scale=sigma)

                current_price = self.data['META_Close'].iloc[i]

                # Cálculo del Stop Loss
                stop_loss = current_price - (beta * (1/sigma) * (1/abs(var_99))) if sigma > 0 and var_99 != 0 else current_price * 0.95

                # Cálculo del Take Profit usando la fórmula proporcionada
                take_profit = current_price + (beta * (1/sigma) * var_99) if sigma > 0 and var_99 != 0 else current_price * 1.05

                if position == 1:
                    if current_price <= stop_loss:
                        position = 0
                        signal = -1
                    elif current_price >= take_profit:
                        position = 0
                        signal = 2  # Nueva señal para take profit alcanzado
                    else:
                        signal = 0
                elif position == 0 and current_price > stop_loss * 1.02 and self.data['META_Returns'].iloc[i-5:i].mean() > 0:
                    position = 1
                    signal = 1
                else:
                    signal = 0

                stops.append(stop_loss)
                take_profits.append(take_profit)
                distances.append((current_price - stop_loss) / current_price * 100)
                signals.append(signal)

            self.data = self.data.iloc[window:].copy()
            self.data['Stop_Loss'] = stops
            self.data['Take_Profit'] = take_profits
            self.data['Distance_Percent'] = distances
            self.data['Signal'] = signals
            self.current_position = position

            print("Stop loss, take profit y señales calculados exitosamente")

        except Exception as e:
            print(f"Error al calcular stop loss y take profit: {str(e)}")
            raise

    def plot_analysis(self):
        """Crea gráficos interactivos con Plotly"""
        if self.data is None:
            raise ValueError("No hay datos para graficar")

        try:
            fig = make_subplots(
                rows=3, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.05,
                subplot_titles=('Precio, Stop Loss y Take Profit', 'Distancia al Stop Loss (%)', 'Volumen'),
                row_heights=[0.5, 0.25, 0.25]
            )

            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['META_Close'], name='META Price', line=dict(color='blue')), row=1, col=1)
            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Stop_Loss'], name='Stop Loss', line=dict(color='red', dash='dash')), row=1, col=1)
            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Take_Profit'], name='Take Profit', line=dict(color='green', dash='dash')), row=1, col=1)

            buy_signals = self.data[self.data['Signal'] == 1]
            sell_signals = self.data[self.data['Signal'] == -1]
            tp_signals = self.data[self.data['Signal'] == 2]

            if not buy_signals.empty:
                fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['META_Close'], name='Buy Signal', mode='markers', marker=dict(color='green', size=10, symbol='triangle-up')), row=1, col=1)

            if not sell_signals.empty:
                fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['META_Close'], name='Sell Signal', mode='markers', marker=dict(color='red', size=10, symbol='triangle-down')), row=1, col=1)

            if not tp_signals.empty:
                fig.add_trace(go.Scatter(x=tp_signals.index, y=tp_signals['META_Close'], name='Take Profit Hit', mode='markers', marker=dict(color='purple', size=10, symbol='star')), row=1, col=1)

            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Distance_Percent'], name='Distance to Stop Loss', line=dict(color='orange')), row=2, col=1)
            fig.add_trace(go.Bar(x=self.data.index, y=self.data['META_Volume'], name='Volume', marker_color='lightblue'), row=3, col=1)

            fig.update_layout(height=1000, title_text="META Analysis Dashboard", showlegend=True)
            return fig

        except Exception as e:
            print(f"Error al crear gráficos: {str(e)}")
            raise

    def get_current_signals(self):
        """Obtiene señales actuales y métricas relevantes"""
        if self.data is None:
            raise ValueError("No hay datos para analizar")

        current_price = self.data['META_Close'].iloc[-1]
        current_stop = self.data['Stop_Loss'].iloc[-1]
        current_tp = self.data['Take_Profit'].iloc[-1]
        current_distance = self.data['Distance_Percent'].iloc[-1]

        metrics = {
            'Precio Actual': f"${current_price:.2f}",
            'Stop Loss': f"${current_stop:.2f}",
            'Take Profit': f"${current_tp:.2f}",
            'Distancia al Stop (%)': f"{current_distance:.2f}%",
            'Distancia al TP (%)': f"{((current_tp - current_price) / current_price * 100):.2f}%",
            'Posición Actual': 'LONG' if self.current_position == 1 else 'FUERA',
            'Señal': self.get_signal_description(),
            'Volatilidad 20d': f"{self.data['META_Returns'].tail(20).std() * np.sqrt(252) * 100:.2f}%",
            'Tendencia 5d': 'ALCISTA' if self.data['META_Returns'].tail(5).mean() > 0 else 'BAJISTA'
        }

        return metrics

    def get_signal_description(self):
        """Genera descripción detallada de la señal actual"""
        if self.data is None:
            raise ValueError("No hay datos para analizar")

        last_signal = self.data['Signal'].iloc[-1]
        if last_signal == 1:
            return "COMPRAR: Precio por encima del stop loss con tendencia positiva"
        elif last_signal == -1:
            return "VENDER: Precio ha tocado el stop loss"
        elif last_signal == 2:
            return "VENDER: Take Profit alcanzado"
        else:
            return "MANTENER: Entre Stop Loss y Take Profit" if self.current_position == 1 else "ESPERAR: Condiciones de entrada no cumplidas"

# Ejemplo de uso
def main():
    try:
        print("Iniciando análisis de META...")
        analyzer = MetaTradeAnalyzer(years=5)

        print("Descargando datos...")
        analyzer.fetch_data()

        print("Calculando señales...")
        analyzer.calculate_stop_loss_and_take_profit()

        print("\n=== Señales Actuales ===")
        current_signals = analyzer.get_current_signals()
        for metric, value in current_signals.items():
            print(f"{metric}: {value}")

        print("\nGenerando gráficos...")
        fig = analyzer.plot_analysis()
        fig.show()

    except Exception as e:
        print(f"Error en la ejecución principal: {str(e)}")

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed

Iniciando análisis de META...
Descargando datos...



[*********************100%***********************]  1 of 1 completed


Datos cargados exitosamente. Rango de fechas: 2019-11-05 00:00:00+00:00 a 2024-11-01 00:00:00+00:00
Calculando señales...
Stop loss, take profit y señales calculados exitosamente

=== Señales Actuales ===
Precio Actual: $567.16
Stop Loss: $538.80
Take Profit: $595.52
Distancia al Stop (%): 5.00%
Distancia al TP (%): 5.00%
Posición Actual: LONG
Señal: MANTENER: Entre Stop Loss y Take Profit
Volatilidad 20d: 25.02%
Tendencia 5d: BAJISTA

Generando gráficos...


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class MetaTradeAnalyzer:
    def __init__(self, years=5):
        self.years = years
        self.end_date = datetime.now()
        self.start_date = self.end_date - timedelta(days=years*365)
        self.data = None
        self.signals = None
        self.current_position = None
        self.last_signal_price = None
        self.last_signal_type = None

    def fetch_data(self):
        """Obtiene datos de META y QQQ"""
        try:
            meta = yf.download('META', start=self.start_date, end=self.end_date)
            qqq = yf.download('QQQ', start=self.start_date, end=self.end_date)

            if len(meta) == 0 or len(qqq) == 0:
                raise ValueError("No se pudieron obtener datos para META o QQQ")

            common_idx = meta.index.intersection(qqq.index)
            meta = meta.loc[common_idx]
            qqq = qqq.loc[common_idx]

            self.data = pd.DataFrame(index=common_idx)
            self.data['META_Close'] = meta['Adj Close']
            self.data['QQQ_Close'] = qqq['Adj Close']
            self.data['META_Volume'] = meta['Volume']
            self.data['META_High'] = meta['High']
            self.data['META_Low'] = meta['Low']

            self.data['META_Returns'] = self.data['META_Close'].pct_change()
            self.data['QQQ_Returns'] = self.data['QQQ_Close'].pct_change()

            print(f"Datos cargados exitosamente. Rango de fechas: {self.data.index[0]} a {self.data.index[-1]}")

        except Exception as e:
            print(f"Error al obtener datos: {str(e)}")
            raise

    def calculate_stop_loss_and_take_profit(self, window=20, confidence_level=0.95):
        """Calcula stop loss y take profit dinámicos y señales"""
        if self.data is None:
            raise ValueError("Debe ejecutar fetch_data() primero")

        try:
            meta_returns = self.data['META_Returns'].dropna()
            qqq_returns = self.data['QQQ_Returns'].dropna()
            beta = np.cov(meta_returns, qqq_returns)[0,1] / np.var(qqq_returns)

            stops, take_profits, distances, signals, positions, entry_prices = [], [], [], [], [], []
            position = 0  # 0: Fuera del mercado, 1: Long
            entry_price = None

            for i in range(window, len(self.data)):
                window_prices = self.data['META_Close'].iloc[i-window:i]
                returns = np.log(window_prices[1:] / window_prices[:-1])
                sigma = np.std(returns)

                # Cálculo del VaR al 99%
                var_99 = stats.norm.ppf(0.99, loc=np.mean(returns), scale=sigma)

                current_price = self.data['META_Close'].iloc[i]

                # Cálculo del Stop Loss y Take Profit
                stop_loss = current_price - (beta * (1/sigma) * (1/abs(var_99))) if sigma > 0 and var_99 != 0 else current_price * 0.95
                take_profit = current_price + (beta * (1/sigma) * var_99) if sigma > 0 and var_99 != 0 else current_price * 1.05

                # Análisis de tendencia
                short_trend = self.data['META_Returns'].iloc[i-5:i].mean() > 0
                momentum = self.data['META_Volume'].iloc[i] > self.data['META_Volume'].iloc[i-5:i].mean()

                # Lógica de trading mejorada
                signal = 0
                if position == 0:  # Si estamos fuera del mercado
                    if short_trend and momentum and current_price > stop_loss * 1.02:
                        signal = 1  # Señal de compra
                        position = 1
                        entry_price = current_price
                else:  # Si estamos en posición long
                    if current_price <= stop_loss:
                        signal = -1  # Señal de venta por stop loss
                        position = 0
                        entry_price = None
                    elif current_price >= take_profit:
                        signal = 2  # Señal de venta por take profit
                        position = 0
                        entry_price = None

                stops.append(stop_loss)
                take_profits.append(take_profit)
                distances.append((current_price - stop_loss) / current_price * 100)
                signals.append(signal)
                positions.append(position)
                entry_prices.append(entry_price)

            self.data = self.data.iloc[window:].copy()
            self.data['Stop_Loss'] = stops
            self.data['Take_Profit'] = take_profits
            self.data['Distance_Percent'] = distances
            self.data['Signal'] = signals
            self.data['Position'] = positions
            self.data['Entry_Price'] = entry_prices
            self.current_position = position

            if entry_price:
                self.last_signal_price = entry_price
                self.last_signal_type = 'COMPRA'
            else:
                self.last_signal_price = current_price
                self.last_signal_type = 'VENTA'

            print("Stop loss, take profit y señales calculados exitosamente")

        except Exception as e:
            print(f"Error al calcular stop loss y take profit: {str(e)}")
            raise

    def calculate_trade_metrics(self):
        """Calcula métricas de trading para la posición actual"""
        if self.data is None or self.current_position is None:
            return None

        current_price = self.data['META_Close'].iloc[-1]
        current_stop = self.data['Stop_Loss'].iloc[-1]
        current_tp = self.data['Take_Profit'].iloc[-1]

        if self.current_position == 1 and self.last_signal_price:
            unrealized_pnl = (current_price - self.last_signal_price) / self.last_signal_price * 100
            risk = (self.last_signal_price - current_stop) / self.last_signal_price * 100
            reward = (current_tp - self.last_signal_price) / self.last_signal_price * 100
            risk_reward_ratio = abs(reward / risk) if risk != 0 else 0
        else:
            unrealized_pnl = 0
            risk = 0
            reward = 0
            risk_reward_ratio = 0

        return {
            'unrealized_pnl': unrealized_pnl,
            'risk': risk,
            'reward': reward,
            'risk_reward_ratio': risk_reward_ratio
        }

    def get_trading_recommendation(self):
        """Genera una recomendación detallada de trading"""
        if self.data is None:
            return "No hay datos suficientes para generar una recomendación"

        current_price = self.data['META_Close'].iloc[-1]
        current_stop = self.data['Stop_Loss'].iloc[-1]
        current_tp = self.data['Take_Profit'].iloc[-1]
        short_trend = self.data['META_Returns'].tail(5).mean() > 0
        momentum = self.data['META_Volume'].iloc[-1] > self.data['META_Volume'].tail(5).mean()

        metrics = self.calculate_trade_metrics()

        recommendation = "=== RECOMENDACIÓN DE TRADING ===\n"

        if self.current_position == 1:
            recommendation += f"POSICIÓN ACTUAL: LONG desde ${self.last_signal_price:.2f}\n"
            recommendation += f"P&L No Realizado: {metrics['unrealized_pnl']:.2f}%\n"
            recommendation += f"Riesgo Actual: {metrics['risk']:.2f}%\n"
            recommendation += f"Recompensa Potencial: {metrics['reward']:.2f}%\n"
            recommendation += f"Ratio Riesgo/Recompensa: {metrics['risk_reward_ratio']:.2f}\n\n"

            if current_price <= current_stop:
                recommendation += "ACCIÓN: VENDER - Stop Loss alcanzado\n"
            elif current_price >= current_tp:
                recommendation += "ACCIÓN: VENDER - Take Profit alcanzado\n"
            else:
                recommendation += "ACCIÓN: MANTENER - Posición dentro de parámetros\n"
                recommendation += f"• Stop Loss en: ${current_stop:.2f}\n"
                recommendation += f"• Take Profit en: ${current_tp:.2f}\n"
        else:
            recommendation += "POSICIÓN ACTUAL: FUERA DEL MERCADO\n"
            if short_trend and momentum and current_price > current_stop * 1.02:
                recommendation += "ACCIÓN: COMPRAR - Condiciones favorables\n"
                recommendation += f"• Entrada sugerida: ${current_price:.2f}\n"
                recommendation += f"• Stop Loss inicial: ${current_stop:.2f}\n"
                recommendation += f"• Take Profit objetivo: ${current_tp:.2f}\n"
                recommendation += f"• Ratio Riesgo/Recompensa: {abs((current_tp - current_price)/(current_price - current_stop)):.2f}\n"
            else:
                recommendation += "ACCIÓN: ESPERAR - Condiciones no favorables\n"
                if not short_trend:
                    recommendation += "• Tendencia corta negativa\n"
                if not momentum:
                    recommendation += "• Volumen insuficiente\n"

        return recommendation

    def plot_analysis(self):
        """Crea gráficos interactivos con Plotly"""
        if self.data is None:
            raise ValueError("No hay datos para graficar")

        try:
            fig = make_subplots(
                rows=3, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.05,
                subplot_titles=('Precio, Stop Loss y Take Profit', 'Distancia al Stop Loss (%)', 'Volumen'),
                row_heights=[0.5, 0.25, 0.25]
            )

            # Gráfico principal con precio y niveles
            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['META_Close'],
                                   name='META Price', line=dict(color='blue')), row=1, col=1)
            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Stop_Loss'],
                                   name='Stop Loss', line=dict(color='red', dash='dash')), row=1, col=1)
            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Take_Profit'],
                                   name='Take Profit', line=dict(color='green', dash='dash')), row=1, col=1)

            # Señales de trading
            buy_signals = self.data[self.data['Signal'] == 1]
            stop_loss_signals = self.data[self.data['Signal'] == -1]
            take_profit_signals = self.data[self.data['Signal'] == 2]

            if not buy_signals.empty:
                fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['META_Close'],
                                       name='Señal de Compra', mode='markers',
                                       marker=dict(color='green', size=10, symbol='triangle-up')), row=1, col=1)

            if not stop_loss_signals.empty:
                fig.add_trace(go.Scatter(x=stop_loss_signals.index, y=stop_loss_signals['META_Close'],
                                       name='Stop Loss Hit', mode='markers',
                                       marker=dict(color='red', size=10, symbol='triangle-down')), row=1, col=1)

            if not take_profit_signals.empty:
                fig.add_trace(go.Scatter(x=take_profit_signals.index, y=take_profit_signals['META_Close'],
                                       name='Take Profit Hit', mode='markers',
                                       marker=dict(color='purple', size=10, symbol='star')), row=1, col=1)

            # Gráfico de distancia al Stop Loss
            fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Distance_Percent'],
                                   name='Distancia al Stop Loss', line=dict(color='orange')), row=2, col=1)

            # Gráfico de volumen
            fig.add_trace(go.Bar(x=self.data.index, y=self.data['META_Volume'],
                               name='Volumen', marker_color='lightblue'), row=3, col=1)

            # Actualizar diseño
            fig.update_layout(
                height=1000,
                title_text="META Analysis Dashboard",
                showlegend=True,
                annotations=[
                    dict(
                        text=self.get_trading_recommendation().replace('\n', '<br>'),
                        align='left',
                        showarrow=False,
                        xref='paper',
                        yref='paper',
                        x=1.02,
                        y=0.5,
                        bordercolor='black',
                        borderwidth=1,
                        bgcolor='white',
                        font=dict(size=10)
                    )
                ]
            )

            return fig

        except Exception as e:
            print(f"Error al crear gráficos: {str(e)}")
            raise

    def get_current_signals(self):
        """Obtiene señales actuales y métricas relevantes"""
        if self.data is None:
            raise ValueError("No hay datos para analizar")

        current_price = self.data['META_Close'].iloc[-1]
        current_stop = self.data['Stop_Loss'].iloc[-1]
        current_tp = self.data['Take_Profit'].iloc[-1]
        metrics = self.calculate_trade_metrics()

        signals = {
            'Precio Actual': f"${current_price:.2f}",
            'Stop Loss': f"${current_stop:.2f}",
            'Take Profit': f"${current_tp:.2f}",
            'Distancia al Stop (%)': f"{self.data['Distance_Percent'].iloc[-1]:.2f}%",
            'Distancia al TP (%)': f"{((current_tp - current_price) / current_price * 100):.2f}%",
            'Posición Actual': 'LONG' if self.current_position == 1 else 'FUERA DEL MERCADO',
            'P&L No Realizado': f"{metrics['unrealized_pnl']:.2f}%" if self.current_position == 1 else "N/A",
            'Ratio Riesgo/Recompensa': f"{metrics['risk_reward_ratio']:.2f}" if self.current_position == 1 else "N/A",
            'Volatilidad 20d': f"{self.data['META_Returns'].tail(20).std() * np.sqrt(252) * 100:.2f}%",
            'Tendencia 5d': 'ALCISTA' if self.data['META_Returns'].tail(5).mean() > 0 else 'BAJISTA',
            'Señal': self.get_trading_recommendation()
        }

        return signals

def main():
    try:
        print("=== INICIANDO ANÁLISIS DE META ===")
        analyzer = MetaTradeAnalyzer(years=5)

        print("\nDescargando datos históricos...")
        analyzer.fetch_data()

        print("\nCalculando señales y niveles...")
        analyzer.calculate_stop_loss_and_take_profit()

        print("\n=== ANÁLISIS DE TRADING ===")
        current_signals = analyzer.get_current_signals()

        # Imprimir métricas principales
        for metric in ['Precio Actual', 'Stop Loss', 'Take Profit', 'Posición Actual', 'P&L No Realizado']:
            print(f"{metric}: {current_signals[metric]}")

        print("\n=== MÉTRICAS TÉCNICAS ===")
        for metric in ['Distancia al Stop (%)', 'Distancia al TP (%)', 'Ratio Riesgo/Recompensa', 'Volatilidad 20d', 'Tendencia 5d']:
            print(f"{metric}: {current_signals[metric]}")

        print("\n=== RECOMENDACIÓN DE TRADING ===")
        print(current_signals['Señal'])

        print("\nGenerando gráfico de análisis...")
        fig = analyzer.plot_analysis()
        fig.show()

        print("\nAnálisis completado exitosamente.")

    except Exception as e:
        print(f"\nError en la ejecución principal: {str(e)}")
        raise

if __name__ == "__main__":
    main()

=== INICIANDO ANÁLISIS DE META ===

Descargando datos históricos...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Datos cargados exitosamente. Rango de fechas: 2019-11-05 00:00:00+00:00 a 2024-11-01 00:00:00+00:00

Calculando señales y niveles...
Stop loss, take profit y señales calculados exitosamente

=== ANÁLISIS DE TRADING ===
Precio Actual: $567.16
Stop Loss: $538.80
Take Profit: $595.52
Posición Actual: LONG
P&L No Realizado: 189.13%

=== MÉTRICAS TÉCNICAS ===
Distancia al Stop (%): 5.00%
Distancia al TP (%): 5.00%
Ratio Riesgo/Recompensa: 1.17
Volatilidad 20d: 25.02%
Tendencia 5d: BAJISTA

=== RECOMENDACIÓN DE TRADING ===
=== RECOMENDACIÓN DE TRADING ===
POSICIÓN ACTUAL: LONG desde $196.16
P&L No Realizado: 189.13%
Riesgo Actual: -174.68%
Recompensa Potencial: 203.59%
Ratio Riesgo/Recompensa: 1.17

ACCIÓN: MANTENER - Posición dentro de parámetros
• Stop Loss en: $538.80
• Take Profit en: $595.52


Generando gráfico de análisis...



Análisis completado exitosamente.


In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class MetaTradeAnalyzer:
    def __init__(self, years=10):
        """
        Inicializa el analizador de trading para META

        Args:
            years (int): Número de años de datos históricos a analizar
        """
        self.years = years
        self.end_date = datetime.now()
        self.start_date = self.end_date - timedelta(days=years*365)
        self.data = None
        self.trades = None  # DataFrame para almacenar detalles de operaciones
        self.current_position = None
        self.last_signal_price = None
        self.last_signal_type = None

    def fetch_data(self):
        """Obtiene datos de META y QQQ"""
        try:
            meta = yf.download('META', start=self.start_date, end=self.end_date)
            qqq = yf.download('QQQ', start=self.start_date, end=self.end_date)

            if len(meta) == 0 or len(qqq) == 0:
                raise ValueError("No se pudieron obtener datos para META o QQQ")

            common_idx = meta.index.intersection(qqq.index)
            meta = meta.loc[common_idx]
            qqq = qqq.loc[common_idx]

            self.data = pd.DataFrame(index=common_idx)
            self.data['META_Close'] = meta['Adj Close']
            self.data['QQQ_Close'] = qqq['Adj Close']
            self.data['META_Volume'] = meta['Volume']
            self.data['META_High'] = meta['High']
            self.data['META_Low'] = meta['Low']

            self.data['META_Returns'] = self.data['META_Close'].pct_change()
            self.data['QQQ_Returns'] = self.data['QQQ_Close'].pct_change()

            print(f"Datos cargados exitosamente. Rango de fechas: {self.data.index[0]} a {self.data.index[-1]}")

        except Exception as e:
            print(f"Error al obtener datos: {str(e)}")
            raise

    def calculate_stop_loss_and_take_profit(self, window=20, confidence_level=0.95):
        """Calcula stop loss y take profit dinámicos y señales"""
        if self.data is None:
            raise ValueError("Debe ejecutar fetch_data() primero")

        try:
            meta_returns = self.data['META_Returns'].dropna()
            qqq_returns = self.data['QQQ_Returns'].dropna()
            beta = np.cov(meta_returns, qqq_returns)[0,1] / np.var(qqq_returns)

            stops, take_profits, signals, positions, entry_prices = [], [], [], [], []
            current_position = 0
            entry_price = None

            for i in range(window, len(self.data)):
                current_price = self.data['META_Close'].iloc[i]
                current_volume = self.data['META_Volume'].iloc[i]
                avg_volume = self.data['META_Volume'].iloc[i-5:i].mean()

                # Calcular tendencias y señales
                short_trend = self.data['META_Returns'].iloc[i-5:i].mean()
                momentum = current_volume > avg_volume

                sma_20 = self.data['META_Close'].iloc[i-20:i].mean()
                sma_50 = self.data['META_Close'].iloc[i-50:i].mean() if i >= 50 else sma_20

                bearish_cross = sma_20 < sma_50 and self.data['META_Close'].iloc[i-1] > sma_20
                high_volume_selloff = current_volume > avg_volume * 1.5 and short_trend < -0.02

                signal = 0
                if current_position == 0:  # Fuera del mercado
                    if short_trend > 0 and momentum and current_price > sma_20:
                        signal = 1  # Señal de compra
                        current_position = 1
                        entry_price = current_price
                    elif bearish_cross or high_volume_selloff:
                        signal = -1  # Señal de venta corta
                        current_position = -1
                        entry_price = current_price

                elif current_position == 1:  # En posición larga
                    if bearish_cross or high_volume_selloff:
                        signal = -1  # Cerrar largo y abrir corto
                        current_position = -1
                        entry_price = current_price

                elif current_position == -1:  # En posición corta
                    if short_trend > 0 and momentum and current_price > sma_20:
                        signal = 1  # Cerrar corto y abrir largo
                        current_position = 1
                        entry_price = current_price

                # Calcular stop loss y take profit según la posición
                if current_position == 1:
                    stop_loss = current_price * 0.95
                    take_profit = current_price * 1.05
                elif current_position == -1:
                    stop_loss = current_price * 1.05
                    take_profit = current_price * 0.95
                else:
                    stop_loss = current_price * 0.95
                    take_profit = current_price * 1.05

                stops.append(stop_loss)
                take_profits.append(take_profit)
                signals.append(signal)
                positions.append(current_position)
                entry_prices.append(entry_price)

            self.data = self.data.iloc[window:].copy()
            self.data['Stop_Loss'] = stops
            self.data['Take_Profit'] = take_profits
            self.data['Signal'] = signals
            self.data['Position'] = positions
            self.data['Entry_Price'] = entry_prices

            self.current_position = current_position
            if entry_price:
                self.last_signal_price = entry_price
                self.last_signal_type = 'COMPRA' if current_position == 1 else 'VENTA'

            print("Señales calculadas exitosamente")

        except Exception as e:
            print(f"Error al calcular señales: {str(e)}")
            raise

    def run_backtest(self):
        """Ejecuta un backtest de la estrategia y muestra los resultados"""
        if self.data is None:
            raise ValueError("No hay datos para el backtest")

        initial_capital = 100000
        position_size = 0.1  # 10% del capital por operación
        commission = 0.001  # 0.1% por operación

        self.data['Equity'] = initial_capital
        self.data['Trade_Profit'] = 0.0

        capital = initial_capital
        current_position = 0
        entry_price = None
        trades = []

        for i in range(1, len(self.data)):
            current_price = self.data['META_Close'].iloc[i]
            signal = self.data['Signal'].iloc[i]
            stop_loss = self.data['Stop_Loss'].iloc[i]
            take_profit = self.data['Take_Profit'].iloc[i]

            # Procesar señales y calcular retornos
            if signal != 0:  # Nueva señal
                if current_position != 0 and entry_price is not None:
                    trade_return = (current_price - entry_price) / entry_price * current_position
                    trade_profit = capital * position_size * trade_return * (1 - commission)
                    capital += trade_profit

                    trades.append({
                        'entry_date': self.data.index[i-1],
                        'exit_date': self.data.index[i],
                        'entry_price': entry_price,
                        'exit_price': current_price,
                        'position': current_position,
                        'profit': trade_profit,
                        'return': trade_return
                    })

                current_position = signal
                entry_price = current_price

            elif current_position != 0 and entry_price is not None:
                hit_stop = (current_position == 1 and current_price <= stop_loss) or \
                          (current_position == -1 and current_price >= stop_loss)
                hit_profit = (current_position == 1 and current_price >= take_profit) or \
                            (current_position == -1 and current_price <= take_profit)

                if hit_stop or hit_profit:
                    trade_return = (current_price - entry_price) / entry_price * current_position
                    trade_profit = capital * position_size * trade_return * (1 - commission)
                    capital += trade_profit

                    trades.append({
                        'entry_date': self.data.index[i-1],
                        'exit_date': self.data.index[i],
                        'entry_price': entry_price,
                        'exit_price': current_price,
                        'position': current_position,
                        'profit': trade_profit,
                        'return': trade_return
                    })

                    current_position = 0
                    entry_price = None

            self.data['Equity'].iloc[i] = capital

        self.trades = pd.DataFrame(trades)
        return self.trades

    def print_metrics(self):
        """Imprime métricas de desempeño de las operaciones realizadas"""
        if self.trades is None or len(self.trades) == 0:
            print("No se han realizado operaciones.")
            return

        total_trades = len(self.trades)
        winning_trades = len(self.trades[self.trades['profit'] > 0])
        losing_trades = len(self.trades[self.trades['profit'] < 0])
        total_profit = self.trades['profit'].sum()
        avg_profit_per_trade = self.trades['profit'].mean()
        success_rate = winning_trades / total_trades * 100

        print("---- Estadísticas de Backtest ----")
        print(f"Número total de operaciones: {total_trades}")
        print(f"Número de operaciones ganadoras: {winning_trades}")
        print(f"Número de operaciones perdedoras: {losing_trades}")
        print(f"Ganancia/Pérdida total: ${total_profit:.2f}")
        print(f"Promedio de ganancia/pérdida por operación: ${avg_profit_per_trade:.2f}")
        print(f"Tasa de éxito: {success_rate:.2f}%")
        print("----------------------------------")

    def plot_results(self):
        """Muestra gráficamente el análisis de META"""
        buys = self.data[self.data['Signal'] == 1]
        sells = self.data[self.data['Signal'] == -1]

        fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                            subplot_titles=("Precio de META con Señales", "Equity Curve", "Volumen"),
                            vertical_spacing=0.1)

        # Gráfico de precio de META con señales
        fig.add_trace(
            go.Scatter(x=self.data.index, y=self.data['META_Close'],
                      mode='lines', name='META Precio', line=dict(color='blue')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=buys.index, y=buys['META_Close'],
                      mode='markers', name='Compra',
                      marker=dict(color='green', size=10, symbol='triangle-up')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=sells.index, y=sells['META_Close'],
                      mode='markers', name='Venta',
                      marker=dict(color='red', size=10, symbol='triangle-down')),
            row=1, col=1
        )

        # Gráfico de equity curve
        fig.add_trace(
            go.Scatter(x=self.data.index, y=self.data['Equity'],
                      name='Equity Curve', line=dict(color='purple')),
            row=2, col=1
        )

        # Gráfico de volumen
        fig.add_trace(
            go.Bar(x=self.data.index, y=self.data['META_Volume'],
                   name='Volume', marker=dict(color='gray')),
            row=3, col=1
        )

        fig.update_layout(
            title='Resultados del Análisis de META',
            xaxis_title='Fecha',
            yaxis_title='Precio de META',
            showlegend=True,
            height=800
        )

        fig.show()

# Crear una instancia del analizador y ejecutar los métodos
analyzer = MetaTradeAnalyzer(years=5)
analyzer.fetch_data()
analyzer.calculate_stop_loss_and_take_profit(window=20, confidence_level=0.95)
results = analyzer.run_backtest()
analyzer.print_metrics()
analyzer.plot_results()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Datos cargados exitosamente. Rango de fechas: 2019-11-05 00:00:00+00:00 a 2024-11-01 00:00:00+00:00
Señales calculadas exitosamente
---- Estadísticas de Backtest ----
Número total de operaciones: 92
Número de operaciones ganadoras: 34
Número de operaciones perdedoras: 58
Ganancia/Pérdida total: $21841.50
Promedio de ganancia/pérdida por operación: $237.41
Tasa de éxito: 36.96%
----------------------------------


In [18]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

class MetaTradeAnalyzer:
    def __init__(self, years=10):
        self.years = years
        self.end_date = datetime.now()
        self.start_date = self.end_date - timedelta(days=years*365)
        self.data = None
        self.trades = None
        self.current_position = None
        self.last_signal_price = None
        self.last_signal_type = None

    def fetch_data(self):
        """Obtiene datos de META y QQQ"""
        try:
            meta = yf.download('META', start=self.start_date, end=self.end_date)
            qqq = yf.download('QQQ', start=self.start_date, end=self.end_date)

            if len(meta) == 0 or len(qqq) == 0:
                raise ValueError("No se pudieron obtener datos para META o QQQ")

            common_idx = meta.index.intersection(qqq.index)
            meta = meta.loc[common_idx]
            qqq = qqq.loc[common_idx]

            self.data = pd.DataFrame(index=common_idx)
            self.data['META_Close'] = meta['Adj Close']
            self.data['QQQ_Close'] = qqq['Adj Close']
            self.data['META_Volume'] = meta['Volume']
            self.data['META_High'] = meta['High']
            self.data['META_Low'] = meta['Low']
            self.data['META_Returns'] = self.data['META_Close'].pct_change()
            self.data['QQQ_Returns'] = self.data['QQQ_Close'].pct_change()

            print(f"Datos cargados exitosamente. Rango de fechas: {self.data.index[0]} a {self.data.index[-1]}")

        except Exception as e:
            print(f"Error al obtener datos: {str(e)}")
            raise

    def calculate_stop_loss_and_take_profit(self, window=20):
        """Calcula stop loss y take profit dinámicos y señales"""
        if self.data is None:
            raise ValueError("Debe ejecutar fetch_data() primero")

        try:
            meta_returns = self.data['META_Returns'].dropna()
            qqq_returns = self.data['QQQ_Returns'].dropna()
            beta = np.cov(meta_returns, qqq_returns)[0, 1] / np.var(qqq_returns)

            sigma = meta_returns.std()
            var_99 = np.percentile(meta_returns, 99)
            var_5 = np.percentile(meta_returns, 5)  # VaR al 5% de confianza

            stops, take_profits, signals, positions, entry_prices = [], [], [], [], []
            current_position = 0
            entry_price = None

            for i in range(window, len(self.data)):
                current_price = self.data['META_Close'].iloc[i]
                current_volume = self.data['META_Volume'].iloc[i]
                avg_volume = self.data['META_Volume'].iloc[i-5:i].mean()

                short_trend = self.data['META_Returns'].iloc[i-5:i].mean()
                momentum = current_volume > avg_volume
                sma_20 = self.data['META_Close'].iloc[i-20:i].mean()
                sma_50 = self.data['META_Close'].iloc[i-50:i].mean() if i >= 50 else sma_20

                bearish_cross = sma_20 < sma_50 and self.data['META_Close'].iloc[i-1] > sma_20
                high_volume_selloff = current_volume > avg_volume * 1.5 and short_trend < -0.02

                signal = 0
                stop_loss = None
                take_profit = None

                if current_position == 0:
                    if short_trend > 0 and momentum and current_price > sma_20:
                        signal = 1  # Señal de compra
                        current_position = 1
                        entry_price = current_price
                    elif bearish_cross or high_volume_selloff:
                        signal = -1  # Señal de venta corta
                        current_position = -1
                        entry_price = current_price

                elif current_position == 1:
                    if bearish_cross or high_volume_selloff:
                        signal = -1  # Cerrar largo y abrir corto
                        current_position = -1
                        entry_price = current_price

                elif current_position == -1:
                    if short_trend > 0 and momentum and current_price > sma_20:
                        signal = 1  # Cerrar corto y abrir largo
                        current_position = 1
                        entry_price = current_price

                if current_position == 1:
                    stop_loss = current_price - (beta * (1 / sigma) * abs(var_5))
                    take_profit = current_price + (beta * (1 / sigma) * var_99)
                elif current_position == -1:
                    stop_loss = current_price + (beta * (1 / sigma) * abs(var_5))
                    take_profit = current_price - (beta * (1 / sigma) * var_99)

                stops.append(stop_loss)
                take_profits.append(take_profit)
                signals.append(signal)
                positions.append(current_position)
                entry_prices.append(entry_price)

            self.data = self.data.iloc[window:].copy()
            self.data['Stop_Loss'] = stops
            self.data['Take_Profit'] = take_profits
            self.data['Signal'] = signals
            self.data['Position'] = positions
            self.data['Entry_Price'] = entry_prices

            self.current_position = current_position
            if entry_price:
                self.last_signal_price = entry_price
                self.last_signal_type = 'COMPRA' if current_position == 1 else 'VENTA'

            print("Señales calculadas exitosamente")

        except Exception as e:
            print(f"Error al calcular señales: {str(e)}")
            raise

    def run_backtest(self):
        """Ejecuta un backtest de la estrategia y muestra los resultados"""
        if self.data is None:
            raise ValueError("No hay datos para el backtest")

        initial_capital = 500
        position_size = 1
        commission = 1  # Comisión fija de $1 por operación

        self.data['Equity'] = initial_capital
        self.data['Trade_Profit'] = 0.0

        capital = initial_capital
        current_position = 0
        entry_price = None
        trades = []

        for i in range(1, len(self.data)):
            current_price = self.data['META_Close'].iloc[i]
            signal = self.data['Signal'].iloc[i]
            stop_loss = self.data['Stop_Loss'].iloc[i]
            take_profit = self.data['Take_Profit'].iloc[i]

            if signal != 0:
                if current_position != 0 and entry_price is not None:
                    trade_return = (current_price - entry_price) / entry_price * current_position
                    trade_profit = (capital * position_size * trade_return) - commission
                    capital += trade_profit

                    trades.append({
                        'entry_date': self.data.index[i-1],
                        'exit_date': self.data.index[i],
                        'entry_price': entry_price,
                        'exit_price': current_price,
                        'position': current_position,
                        'profit': trade_profit,
                        'return': trade_return
                    })

                current_position = signal
                entry_price = current_price

            elif current_position != 0 and entry_price is not None:
                hit_stop = (current_position == 1 and current_price <= stop_loss) or \
                          (current_position == -1 and current_price >= stop_loss)
                hit_profit = (current_position == 1 and current_price >= take_profit) or \
                            (current_position == -1 and current_price <= take_profit)

                if hit_stop or hit_profit:
                    trade_return = (current_price - entry_price) / entry_price * current_position
                    trade_profit = (capital * position_size * trade_return) - commission
                    capital += trade_profit

                    trades.append({
                        'entry_date': self.data.index[i-1],
                        'exit_date': self.data.index[i],
                        'entry_price': entry_price,
                        'exit_price': current_price,
                        'position': current_position,
                        'profit': trade_profit,
                        'return': trade_return
                    })

                    current_position = 0
                    entry_price = None

            self.data['Equity'].iloc[i] = capital

        self.trades = pd.DataFrame(trades)
        return self.trades

    def print_metrics(self):
        """Imprime métricas de desempeño de las operaciones realizadas"""
        if self.trades is not None and not self.trades.empty:
            print(self.trades)
            print(f"Capital final: ${self.data['Equity'].iloc[-1]:.2f}")
            print(f"Número de operaciones: {len(self.trades)}")
            print(f"Ganancias/perdidas promedio por operación: ${self.trades['profit'].mean():.2f}")
        else:
            print("No se han realizado operaciones.")

# Ejecución del análisis
analyzer = MetaTradeAnalyzer(years=10)
analyzer.fetch_data()
analyzer.calculate_stop_loss_and_take_profit()
trades = analyzer.run_backtest()
analyzer.print_metrics()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Datos cargados exitosamente. Rango de fechas: 2014-11-06 00:00:00+00:00 a 2024-11-01 00:00:00+00:00
Señales calculadas exitosamente
                   entry_date                 exit_date  entry_price  \
0   2015-01-29 00:00:00+00:00 2015-01-30 00:00:00+00:00    76.609337   
1   2015-02-17 00:00:00+00:00 2015-02-18 00:00:00+00:00    75.682137   
2   2015-02-18 00:00:00+00:00 2015-02-19 00:00:00+00:00    76.479729   
3   2015-02-19 00:00:00+00:00 2015-02-20 00:00:00+00:00    79.181595   
4   2015-02-20 00:00:00+00:00 2015-02-23 00:00:00+00:00    79.660156   
..                        ...                       ...          ...   
179 2024-06-14 00:00:00+00:00 2024-06-17 00:00:00+00:00   503.120026   
180 2024-08-01 00:00:00+00:00 2024-08-02 00:00:00+00:00   506.147156   
181 2024-08-14 00:00:00+00:00 2024-08-15 00:00:00+00:00   487.674774   
182 2024-08-15 00:00:00+00:00 2024-08-16 00:00:00+00:00   536.817871   
183 2024-08-21 00:00:00+00:00 2024-08-22 00:00:00+00:00   526.917297   

   

In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class MetaTradeAnalyzer:
    def __init__(self, years=10):
        self.years = years
        self.end_date = datetime.now()
        self.start_date = self.end_date - timedelta(days=years*365)
        self.data = None
        self.trades = None
        self.current_position = None
        self.last_signal_price = None
        self.last_signal_type = None

    def fetch_data(self):
        """Obtiene datos de META y QQQ"""
        try:
            meta = yf.download('META', start=self.start_date, end=self.end_date)
            qqq = yf.download('QQQ', start=self.start_date, end=self.end_date)

            if len(meta) == 0 or len(qqq) == 0:
                raise ValueError("No se pudieron obtener datos para META o QQQ")

            common_idx = meta.index.intersection(qqq.index)
            meta = meta.loc[common_idx]
            qqq = qqq.loc[common_idx]

            self.data = pd.DataFrame(index=common_idx)
            self.data['META_Close'] = meta['Adj Close']
            self.data['QQQ_Close'] = qqq['Adj Close']
            self.data['META_Volume'] = meta['Volume']
            self.data['META_High'] = meta['High']
            self.data['META_Low'] = meta['Low']
            self.data['META_Returns'] = self.data['META_Close'].pct_change()
            self.data['QQQ_Returns'] = self.data['QQQ_Close'].pct_change()

            print(f"Datos cargados exitosamente. Rango de fechas: {self.data.index[0]} a {self.data.index[-1]}")

        except Exception as e:
            print(f"Error al obtener datos: {str(e)}")
            raise

    def calculate_stop_loss_and_take_profit(self, window=20):
        """Calcula stop loss y take profit dinámicos y señales"""
        if self.data is None:
            raise ValueError("Debe ejecutar fetch_data() primero")

        try:
            meta_returns = self.data['META_Returns'].dropna()
            qqq_returns = self.data['QQQ_Returns'].dropna()
            beta = np.cov(meta_returns, qqq_returns)[0, 1] / np.var(qqq_returns)

            sigma = meta_returns.std()
            var_99 = np.percentile(meta_returns, 99)
            var_5 = np.percentile(meta_returns, 5)  # VaR al 5% de confianza

            stops, take_profits, signals, positions, entry_prices = [], [], [], [], []
            current_position = 0
            entry_price = None

            for i in range(window, len(self.data)):
                current_price = self.data['META_Close'].iloc[i]
                current_volume = self.data['META_Volume'].iloc[i]
                avg_volume = self.data['META_Volume'].iloc[i-5:i].mean()

                short_trend = self.data['META_Returns'].iloc[i-5:i].mean()
                momentum = current_volume > avg_volume
                sma_20 = self.data['META_Close'].iloc[i-20:i].mean()
                sma_50 = self.data['META_Close'].iloc[i-50:i].mean() if i >= 50 else sma_20

                bearish_cross = sma_20 < sma_50 and self.data['META_Close'].iloc[i-1] > sma_20
                high_volume_selloff = current_volume > avg_volume * 1.5 and short_trend < -0.02

                signal = 0
                stop_loss = None
                take_profit = None

                if current_position == 0:
                    if short_trend > 0 and momentum and current_price > sma_20:
                        signal = 1  # Señal de compra
                        current_position = 1
                        entry_price = current_price
                    elif bearish_cross or high_volume_selloff:
                        signal = -1  # Señal de venta corta
                        current_position = -1
                        entry_price = current_price

                elif current_position == 1:
                    if bearish_cross or high_volume_selloff:
                        signal = -1  # Cerrar largo y abrir corto
                        current_position = -1
                        entry_price = current_price

                elif current_position == -1:
                    if short_trend > 0 and momentum and current_price > sma_20:
                        signal = 1  # Cerrar corto y abrir largo
                        current_position = 1
                        entry_price = current_price

                if current_position == 1:
                    stop_loss = current_price - (beta * (1 / sigma) * abs(var_5))
                    take_profit = current_price + (beta * (1 / sigma) * var_99)
                elif current_position == -1:
                    stop_loss = current_price + (beta * (1 / sigma) * abs(var_5))
                    take_profit = current_price - (beta * (1 / sigma) * var_99)

                stops.append(stop_loss)
                take_profits.append(take_profit)
                signals.append(signal)
                positions.append(current_position)
                entry_prices.append(entry_price)

            self.data = self.data.iloc[window:].copy()
            self.data['Stop_Loss'] = stops
            self.data['Take_Profit'] = take_profits
            self.data['Signal'] = signals
            self.data['Position'] = positions
            self.data['Entry_Price'] = entry_prices

            self.current_position = current_position
            if entry_price:
                self.last_signal_price = entry_price
                self.last_signal_type = 'COMPRA' if current_position == 1 else 'VENTA'

            print("Señales calculadas exitosamente")

        except Exception as e:
            print(f"Error al calcular señales: {str(e)}")
            raise

    def run_backtest(self):
        """Ejecuta un backtest de la estrategia y muestra los resultados"""
        if self.data is None:
            raise ValueError("No hay datos para el backtest")

        initial_capital = 500
        position_size = 1
        commission = 1  # Comisión fija de $1 por operación

        self.data['Equity'] = initial_capital
        self.data['Trade_Profit'] = 0.0

        capital = initial_capital
        current_position = 0
        entry_price = None
        trades = []

        for i in range(1, len(self.data)):
            current_price = self.data['META_Close'].iloc[i]
            signal = self.data['Signal'].iloc[i]
            stop_loss = self.data['Stop_Loss'].iloc[i]
            take_profit = self.data['Take_Profit'].iloc[i]

            if signal != 0:
                if current_position != 0 and entry_price is not None:
                    trade_return = (current_price - entry_price) / entry_price * current_position
                    trade_profit = (capital * position_size * trade_return) - commission
                    capital += trade_profit

                    trades.append({
                        'entry_date': self.data.index[i-1],
                        'exit_date': self.data.index[i],
                        'entry_price': entry_price,
                        'exit_price': current_price,
                        'position': current_position,
                        'profit': trade_profit,
                        'return': trade_return
                    })

                current_position = signal
                entry_price = current_price

            elif current_position != 0 and entry_price is not None:
                hit_stop = (current_position == 1 and current_price <= stop_loss) or \
                          (current_position == -1 and current_price >= stop_loss)
                hit_profit = (current_position == 1 and current_price >= take_profit) or \
                            (current_position == -1 and current_price <= take_profit)

                if hit_stop or hit_profit:
                    trade_return = (current_price - entry_price) / entry_price * current_position
                    trade_profit = (capital * position_size * trade_return) - commission
                    capital += trade_profit

                    trades.append({
                        'entry_date': self.data.index[i-1],
                        'exit_date': self.data.index[i],
                        'entry_price': entry_price,
                        'exit_price': current_price,
                        'position': current_position,
                        'profit': trade_profit,
                        'return': trade_return
                    })

                    current_position = 0
                    entry_price = None

            self.data['Equity'].iloc[i] = capital

        self.trades = pd.DataFrame(trades)
        return self.trades

    def print_metrics(self):
        """Imprime métricas de desempeño de las operaciones realizadas"""
        if self.trades is not None and not self.trades.empty:
            print(self.trades)
            print(f"Capital final: ${self.data['Equity'].iloc[-1]:.2f}")
            print(f"Número de operaciones: {len(self.trades)}")
            print(f"Ganancias/perdidas promedio por operación: ${self.trades['profit'].mean():.2f}")
        else:
            print("No se han realizado operaciones.")

    def plot_results(self):
        """Genera gráficos de resultados"""
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)

        # Gráfico de capital
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Equity'], name='Capital', line=dict(color='blue')), row=1, col=1)

        # Gráfico de señales, stop loss y take profit
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Stop_Loss'], name='Stop Loss', line=dict(color='red', dash='dash')), row=2, col=1)
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['Take_Profit'], name='Take Profit', line=dict(color='green', dash='dash')), row=2, col=1)
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['META_Close'], name='Precio META', line=dict(color='orange')), row=2, col=1)

        # Añadir marcadores para las señales de trading
        buy_signals = self.data[self.data['Signal'] == 1]
        sell_signals = self.data[self.data['Signal'] == -1]

        fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['META_Close'], mode='markers', name='Señal de Compra', marker=dict(symbol='arrow-up', size=10, color='green')), row=2, col=1)
        fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['META_Close'], mode='markers', name='Señal de Venta', marker=dict(symbol='arrow-down', size=10, color='red')), row=2, col=1)

        # Actualizar el diseño del gráfico
        fig.update_layout(title='Resultados del Análisis de Trading de META', xaxis_title='Fecha', yaxis_title='Capital', height=800)
        fig.show()

# Ejecución del análisis
analyzer = MetaTradeAnalyzer(years=10)
analyzer.fetch_data()
analyzer.calculate_stop_loss_and_take_profit()
trades = analyzer.run_backtest()
analyzer.print_metrics()
analyzer.plot_results()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Datos cargados exitosamente. Rango de fechas: 2014-11-06 00:00:00+00:00 a 2024-11-01 00:00:00+00:00
Señales calculadas exitosamente
                   entry_date                 exit_date  entry_price  \
0   2015-01-29 00:00:00+00:00 2015-01-30 00:00:00+00:00    76.609337   
1   2015-02-17 00:00:00+00:00 2015-02-18 00:00:00+00:00    75.682137   
2   2015-02-18 00:00:00+00:00 2015-02-19 00:00:00+00:00    76.479729   
3   2015-02-19 00:00:00+00:00 2015-02-20 00:00:00+00:00    79.181595   
4   2015-02-20 00:00:00+00:00 2015-02-23 00:00:00+00:00    79.660149   
..                        ...                       ...          ...   
179 2024-06-14 00:00:00+00:00 2024-06-17 00:00:00+00:00   503.120026   
180 2024-08-01 00:00:00+00:00 2024-08-02 00:00:00+00:00   506.147156   
181 2024-08-14 00:00:00+00:00 2024-08-15 00:00:00+00:00   487.674774   
182 2024-08-15 00:00:00+00:00 2024-08-16 00:00:00+00:00   536.817871   
183 2024-08-21 00:00:00+00:00 2024-08-22 00:00:00+00:00   526.917297   

   